In [ ]:
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import List

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [26]:
class InputType(Enum):
    Touch = 1
    TUI = 2
    
class Metaphor(Enum):
    Gesture = 1
    Joystick = 2
    Car =  3

@dataclass
class GpxTrack:
    track_id: int
    user_id: int
    input_type: InputType
    metaphor: Metaphor
    file: Path
    
    def __init__(self, file_path: Path):
        file_name = file_path.stem
        parts = file_name.split("_")
        self.track_id: int = int(parts[0])
        self.user_id: int = int(parts[1])
        self.input_type: InputType = InputType[parts[2]]
        self.metaphor: Metaphor = Metaphor[parts[3]]
        self.file: Path = file_path
        
@dataclass
class TrackRepository:
    reference_track_files: List[Path]
    recorded_tracks: List[GpxTrack]
    
    def __init__(self):
        self.reference_track_files = [track_file for track_file in Path("reference_tracks").iterdir() if track_file.is_file()]
        self.recorded_tracks = [GpxTrack(track_file) for track_file in Path("recorded_tracks").iterdir() if track_file.is_file()]
        
    def get_by_track(self, track_id: int) -> List[GpxTrack]:
        return [track for track in self.recorded_tracks if track.track_id == track_id]

    def get_by_user(self, user_id: int) -> List[GpxTrack]:
        return [track for track in self.recorded_tracks if track.user_id == user_id]
    
    def get_by_input_type(self, input_type: InputType) -> List[GpxTrack]:
        return [track for track in self.recorded_tracks if track.input_type == input_type]
    
    def get_by_metaphor(self, metaphor: Metaphor) -> List[GpxTrack]:
        return [track for track in self.recorded_tracks if track.metaphor == metaphor]
    

In [27]:
repo = TrackRepository()
print(repo.get_by_track(1))

[GpxTrack(track_id=1, user_id=1, input_type=<InputType.Touch: 1>, metaphor=<Metaphor.Gesture: 1>, file=WindowsPath('recorded_tracks/1_1_Touch_Gesture_24-11-01-11-32-42.gpx')), GpxTrack(track_id=1, user_id=1, input_type=<InputType.Touch: 1>, metaphor=<Metaphor.Joystick: 2>, file=WindowsPath('recorded_tracks/1_1_Touch_Joystick_24-11-01-12-00-54.gpx')), GpxTrack(track_id=1, user_id=1, input_type=<InputType.TUI: 2>, metaphor=<Metaphor.Car: 3>, file=WindowsPath('recorded_tracks/1_1_TUI_Car_24-11-01-12-12-41.gpx')), GpxTrack(track_id=1, user_id=1, input_type=<InputType.TUI: 2>, metaphor=<Metaphor.Joystick: 2>, file=WindowsPath('recorded_tracks/1_1_TUI_Joystick_24-11-01-12-26-28.gpx'))]


In [ ]:
tracks = {}
for ref in reference_track_files:
    name = ref.stem
    tracks[ref] = [track for track in recorded_track_files if str(track.stem).split("_")[0] == name]

In [ ]:
from gps_accuracy.gps_accuracy import GpxEvaluator
results = {}
for k,v in tracks.items():
    name = k.stem
    track_results = []
    for track_file in v:
        evaluator = GpxEvaluator(k, track_file)
        track_results.append(evaluator.evaluate())
    results[name] = track_results

In [ ]:
track_names = [name for name in results.keys()]
mean_errors = {
    "Touch_Gesture" : [results[track_name][0].mean for track_name in results.keys()],
    "Touch_Joystick" : [results[track_name][1].mean for track_name in results.keys()],
    "TUI_Car": [results[track_name][2].mean for track_name in results.keys()],
    "TUI_Joystick": [results[track_name][3].mean for track_name in results.keys()]
}

max_value = np.max([x for x in mean_errors.values()])
max_value *= 1.1
x = np.arange(len(track_names))
width = 0.1
multiplier = 0

fig, ax = plt.subplots(layout='compressed')

fig.set_figwidth(10)

for attribute, measurement in mean_errors.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=4)
    multiplier += 1

ax.set_ylabel('Mean Error')
ax.set_title('Mean Error by Input Type')
ax.set_xticks(x + width, track_names)
ax.legend(loc='upper left', ncols=4)
ax.set_ylim(0, max_value)

plt.show()

In [ ]:
track_names = [name for name in results.keys()]
times = {
    "Touch_Gesture" : [results[track_name][0].time for track_name in results.keys()],
    "Touch_Joystick" : [results[track_name][1].time for track_name in results.keys()],
    "TUI_Car": [results[track_name][2].time for track_name in results.keys()],
    "TUI_Joystick": [results[track_name][3].time for track_name in results.keys()]
}

max_value = np.max([x for x in times.values()])
max_value *= 1.1

x = np.arange(len(track_names))
width = 0.1
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

fig.set_figwidth(10)

for attribute, measurement in times.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=4)
    multiplier += 1

ax.set_ylabel('Time')
ax.set_title('Time needed by Input Type in s')
ax.set_xticks(x + width, track_names)
ax.legend(loc='upper left', ncols=4)
ax.set_ylim(0, max_value)

plt.show()